<a href="https://colab.research.google.com/github/FerDoranNie/rag_consar_nerdfore/blob/master/Test/AFORE_rag_consar_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG CONSAR NERDAFORE
En este notebook se probarán todas las funciones del proceso del RAG para NerdAfore.

https://github.com/FerDoranNie/rag_consar_nerdfore




## Instalar librerías

*   Debes subir el documento de los requirements



In [1]:
!playwright install
!sudo apt-get install tesseract-ocr
!sudo apt-get install tesseract-ocr-spa


Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Please install them with the following command:      ║
║                                                      ║
║     playwright install-deps                          ║
║                                                      ║
║ Alternatively, use apt:                              ║
║     apt-get install libatk1.0-0\                     ║
║         libatk-bridge2.0-0\                          ║
║         libatspi2.0-0\                               ║
║         libxcomposite1                               ║
║                                                      ║
║ <3 Playwright Team                                   ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.12/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at async Registry.

In [2]:
!pip install --quiet -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 26.3 MB/s eta 0:00:00


In [3]:
#!pip install pytesseract

## Pipeline principal

### General Functions

In [5]:
import sys
import os
import unicodedata
import re
from pathlib import Path
from typing import Dict, Optional, List
import pandas as pd
import numpy as np
import json
from datetime import datetime, timezone


## Función para remover acentos

def remove_accents(input_str):
    if not isinstance(input_str, str):
        raise TypeError("Input must be a string.")
    normalized = unicodedata.normalize('NFD', input_str)

    without_accents = ''.join(
        char for char in normalized
        if unicodedata.category(char) != 'Mn'
    )
    return without_accents


def clean_string(string_to_clean, with_underscore=False):
  string_cleaned = re.sub(r'_', ' ', string_to_clean)
  ## Cambiar acentos por letras sin acentos, también aplica la ñ
  string_cleaned = remove_accents(string_cleaned)
  ## Remover caracteres raros
  string_cleaned = re.sub(r'[^\w\s]', '', string_cleaned)
  ## Convert to UTF8
  string_cleaned = string_cleaned.encode('utf-8').decode('utf-8')
  string_cleaned = re.sub(r'\s+', ' ', string_cleaned)
  ### Remover espacios al inicio y al final
  string_cleaned = string_cleaned.strip()
  if with_underscore:
    return  string_cleaned.replace(' ', '_').lower()
  else:
    return string_cleaned.lower()



### Funciones de fecha
def get_current_date():
    current_iso_date = datetime.now().date().isoformat()
    return current_iso_date

def get_current_timestamp_utc():
    utc_time = datetime.now(timezone.utc)
    return utc_time.isoformat()

### Funciones de archivos

def get_filelists_from_folder_root(folder_path, file_type):
    ### Función para detectar tipo de archivos
    #### Lo devuelve como lista
    try:
        list_files_detected = [file for file in os.listdir(folder_path) if file.endswith(file_type)]
        return list_files_detected
    except Exception as e:
        print(f'Hay un error al obtener los archivos: {e}')
        return []

def get_file_extension(file_path):
    file_path = Path(file_path)
    file_extension = file_path.suffix
    return file_extension



def get_filelists_from_folder(folder_path, file_type):
    ### Función para detectar tipo de archivos
    #### Lo devuelve como lista
    list_files_detected = []
    try:
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.endswith(file_type):
                    list_files_detected.append(os.path.join(root, file))
        return list_files_detected
    except Exception as e:
        print(f'Hay un error al obtener los archivos: {e}')
        return []

def get_unique_file_extensions(folder_path: str) -> List[str]:
    """
    Busca recursivamente y devuelve una lista de todas las extensiones de archivo
    únicas encontradas en una carpeta y sus subcarpetas.

    Args:
        folder_path (str): La ruta de la carpeta principal a explorar.

    Returns:
        List[str]: Una lista ordenada de extensiones de archivo únicas (ej. ['csv', 'pdf', 'txt']).
                   Devuelve una lista vacía si ocurre un error.
    """
    unique_extensions = set()
    try:
        for _, _, files in os.walk(folder_path):
            for file in files:
                _, ext = os.path.splitext(file)
                if ext:  # Asegurarse de que el archivo tiene una extensión
                    unique_extensions.add(ext.lower().lstrip('.'))
        return sorted(list(unique_extensions))
    except Exception as e:
        print(f'Ocurrió un error al obtener las extensiones de archivo: {e}')
        return []




def analyze_string_project(filename_str, project_siglas): # No sé como se dice siglas en inglés XD
    if not filename_str.startswith(project_siglas):
        raise ValueError(f"Las siglas '{project_siglas}' no coinciden con el inicio de la cadena")

    split_str = filename_str.split('-')
    specific_name = split_str[-1]
    date_pattern = r'(\d{2})-(\d{2})'
    match_date = re.search(date_pattern, filename_str)

    if match_date:
        day = match_date.group(1)
        month = match_date.group(2)
        date_str = f"{day}/{month}"
    else:
        day = month = date_str = "Date not found"

    #pattern_file_version = r'(?<!-)(\d{4})(?!\d{2}[-])'
    pattern_file_version = r'[A-Za-z](\d+)(?=\d{2}-\d{2}-)'
    match_version = re.search(pattern_file_version, filename_str.replace(project_siglas,
                                                                   '', 1))

    if match_version:
        version = match_version.group(1)
    else:
        version = "Version not found"

    string_extra_characters = filename_str[len(project_siglas):]
    pattern_job = r'^([A-Za-z]{2,4})'
    match_job = re.search(pattern_job, string_extra_characters)

    if match_job:
        kind_job = match_job.group(1)
    else:
        kind_job = "Job not found"


    return {
        'cadena_original': filename_str,
        'siglas_proyecto': project_siglas,
        'tipo_trabajo': kind_job,
        'dia': day,
        'mes': month,
        'fecha': day+'-' + month ,
        'numero_version': version,
        'nombre_especifico': specific_name
    }


def analyze_string_with_list(filename_str, list_siglas):
    siglas_found = []
    for siglas in list_siglas:
        if filename_str.startswith(siglas):
            siglas_found.append(siglas)

    if not siglas_found:
        raise ValueError(f'No se encontró ninguna sigla que coincida')

    project_siglas = max(siglas_found, key=len)
    return analyze_string_project(filename_str, project_siglas)



def get_file_category(filename):
    """
    Determina la categoría de un archivo (texto, audio, video, imagen)
    basándose en su extensión.

    Args:
        filename (str): El nombre del archivo (puede incluir la ruta completa).

    Returns:
        str: La categoría del archivo ("texto", "audio", "video", "imagen")
             o "desconocido" si la extensión no coincide con ninguna categoría.
    """
    _, ext = os.path.splitext(filename)
    ext = ext.lower().lstrip('.')

    text_extensions = [
        'txt', 'csv', 'json', 'xml', 'md', 'log', 'py', 'html', 'css', 'js',
        'pdf', 'docx', 'xlsx', 'pptx', 'rtf', 'odt', 'ods', 'odp'
    ]
    audio_extensions = [
        'mp3', 'wav', 'aac', 'flac', 'ogg', 'm4a', 'wma', 'aiff', 'alac'
    ]
    video_extensions = [
        'mp4', 'avi', 'mov', 'mkv', 'wmv', 'flv', 'webm', 'mpeg', 'mpg', '3gp'
    ]
    image_extensions = [
        'jpg', 'jpeg', 'png', 'gif', 'bmp', 'tiff', 'webp', 'svg', 'ico', 'heif', 'heic'
    ]

    if ext in text_extensions:
        return "texto"
    elif ext in audio_extensions:
        return "audio"
    elif ext in video_extensions:
        return "video"
    elif ext in image_extensions:
        return "imagen"
    else:
        return "desconocido"



def get_filename(file_path):
    try:
        file_path = str(file_path)
        regex_to_get= r'(?!.*\\).+'
        x = re.search(regex_to_get, file_path)
        return x.group(0)
    except Exception as e:
        print(f'Error detectado en la cadena {e}')
        return None



def read_json_file(filename_str):

    with open(filename_str, 'r') as file:
        data = json.load(file)

    return data




def complete_dict(
    original_dict,
    list_keys,
    default_value='desconocido'):
    """
    Asegura que un diccionario contenga un conjunto de claves requeridas.

    Recorre una lista de claves. Si una clave existe en el diccionario original,
    mantiene su valor. Si no existe, la añade al nuevo diccionario con un
    valor por defecto.

    Args:
        diccionario_original (Dict[str, Any]): El diccionario de entrada que puede
                                               tener claves faltantes.
        claves_requeridas (List[str]): Una lista de strings con todas las claves
                                       que el diccionario final debe tener.
        valor_por_defecto (Any, optional): El valor que se asignará a las claves
                                           que no se encuentren. Por defecto es "desconocido".

    Returns:
        Dict[str, Any]: Un nuevo diccionario completo con todas las claves requeridas.
    """
    return {
        key: original_dict.get(key, default_value)
        for key in list_keys
    }

### load txt files
def load_text_files(directory):
    texts = {}
    for file in Path(directory).glob("*.txt"):
        with open(file, "r", encoding="utf-8") as f:
            content = f.read().strip()
            if content:
                texts[file.name] = content
    return texts



### Convert audio, video, pdfs to text

In [9]:
import os
import whisper
from pydub import AudioSegment
import speech_recognition as sr
import logging
import pandas as pd
import numpy as np

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Cargando modelo de Whisper 'medium'...")
whisperModel = whisper.load_model('medium')
logging.info("Modelo de Whisper cargado exitosamente.")

def transcribe_audio_array(audio_array: np.ndarray, language: str = "es") -> tuple[str, pd.DataFrame] | None:
    if not isinstance(audio_array, np.ndarray):
        logging.error("La entrada no es un array de NumPy válido.")
        return None
    try:
        logging.info(f"Iniciando transcripción en idioma '{language}'...")
        # El array ya está en el formato que Whisper espera (float32)
        result = whisperModel.transcribe(audio_array, language=language)

        transcribed_text = result['text']
        logging.info("Transcripción completada exitosamente.")
        return transcribed_text
    except Exception as e:
        logging.error(f"Error durante la transcripción con Whisper: {e}")
        return None


def transcribe_audio_file(audio_path: str, language: str = "es") -> str | None:
    if not os.path.exists(audio_path):
        logging.error(f"El archivo de audio no fue encontrado en la ruta: {audio_path}")
        return None

    try:
        logging.info(f"Iniciando transcripción del archivo '{os.path.basename(audio_path)}' en idioma '{language}'...")
        # Whisper puede procesar la ruta del archivo directamente
        result = whisperModel.transcribe(audio_path, language=language)
        transcribed_text = result['text']
        logging.info(f"Transcripción del archivo '{os.path.basename(audio_path)}' completada exitosamente.")
        return transcribed_text
    except Exception as e:
        logging.error(f"Error durante la transcripción del archivo '{audio_path}': {e}")
        return None



def transcribe_video_to_audio_to_text(video_path, language='es'):
    try:
        print('Creando archivo temporal')
        video_to_audio(video_path, 'temp.wav')
        audio_path = 'temp.wav'
        text_from_audio = transcribe_audio_file(audio_path=audio_path, language=language)
        print('Borrando archivo temporal')
        if os.path.isfile(audio_path):
            os.remove(audio_path)
        return text_from_audio
    except Exception as e:
        logging.error(f"Error durante la transcripción del archivo '{video_path}': {e}")
        return None


#### Funciones de video a audio
from moviepy.editor import VideoFileClip
import logging
import numpy as np
from io import BytesIO
from typing import Optional
from scipy.io.wavfile import write as write_wav

# Configurar logging para mostrar mensajes informativos y de error
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def video_to_audio(video_path: str, audio_path: str) -> str | None:
    """
    Extrae el audio de un archivo de video y lo guarda en la ruta especificada.

    Args:
        video_path (str): La ruta al archivo de video de entrada.
        audio_path (str): La ruta donde se guardará el archivo de audio extraído (ej. 'output.mp3').

    Returns:
        Optional[str]: La ruta al archivo de audio creado si la conversión es exitosa,
                    en caso contrario, None.
    """
    video_clip = None
    try:
        logging.info(f"Iniciando extracción de audio para: {video_path}")
        video_clip = VideoFileClip(video_path)
        audio_clip = video_clip.audio

        # Escribir el archivo de audio. Moviepy deducirá el códec del nombre del archivo.
        audio_clip.write_audiofile(audio_path)

        logging.info(f"Audio guardado exitosamente en: {audio_path}")
        return audio_path
    except Exception as e:
        logging.error(f"Error al convertir el video a audio para '{video_path}': {e}")
        return None
    finally:
        # Asegurarse de cerrar los clips para liberar recursos
        if video_clip and video_clip.audio:
            audio_clip.close()
        if video_clip:
            video_clip.close()


def extract_audio_for_transcription(video_path: str) -> np.ndarray | None:
    """
    Extrae el audio de un video y lo devuelve como un array NumPy,
    listo para ser procesado por modelos como Whisper.

    El audio se convierte a mono y se remuestrea a 16kHz.

    Args:
        video_path (str): La ruta al archivo de video de entrada.

    Returns:
        Optional[np.ndarray]: Un array NumPy con la forma de onda del audio si es exitoso,
                          en caso contrario, None.
    """
    video_clip = None
    audio_clip = None
    try:
        logging.info(f"Extrayendo audio en memoria desde: {video_path}")
        video_clip = VideoFileClip(video_path)
        audio_clip = video_clip.audio

        # Convertir el audio a un array NumPy, remuestreando a 16kHz (estándar para Whisper)
        # El resultado es un array flotante, que es lo que Whisper espera.
        audio_waveform = audio_clip.to_soundarray(fps=16000)

        # Asegurarse de que el audio sea mono (un solo canal)
        if audio_waveform.ndim > 1 and audio_waveform.shape[1] == 2:
            audio_waveform = audio_waveform.mean(axis=1)

        logging.info(f"Audio extraído en memoria exitosamente.")
        return audio_waveform.astype(np.float32)
    except Exception as e:
        logging.error(f"Error al extraer el audio en memoria para '{video_path}': {e}")
        return None
    finally:
        if audio_clip:
            audio_clip.close()
        if video_clip:
            video_clip.close()


def video_to_audio_buffer(video_path: str) -> Optional[BytesIO]:
    """
    Extrae el audio de un video y lo guarda en un buffer en memoria como formato WAV.

    Este buffer es ideal para ser consumido directamente por librerías de transcripción
    como Whisper, evitando la creación de archivos temporales en disco.

    Args:
        video_path (str): La ruta al archivo de video de entrada.

    Returns:
        Optional[BytesIO]: Un objeto BytesIO con el audio en formato WAV si es exitoso,
                           en caso contrario, None.
    """
    video_clip = None
    audio_clip = None
    try:
        logging.info(f"Iniciando extracción de audio a buffer desde: {video_path}")
        video_clip = VideoFileClip(video_path)
        audio_clip = video_clip.audio

        audio_buffer = BytesIO()
        # 1. Extraer la forma de onda del audio a 16kHz.
        audio_waveform = audio_clip.to_soundarray(fps=16000)

        # 2. Asegurarse de que el audio sea mono.
        if audio_waveform.ndim > 1 and audio_waveform.shape[1] == 2:
            audio_waveform = audio_waveform.mean(axis=1)

        # 3. Convertir la forma de onda de float a int16, que es el formato estándar para WAV (pcm_s16le).
        # Whisper internamente maneja float32, pero para un archivo WAV estándar, int16 es más común.
        # Multiplicamos por el valor máximo de int16 para escalar la señal.
        audio_int16 = (audio_waveform * 32767).astype(np.int16)

        # 4. Escribir el array NumPy en el buffer en formato WAV usando SciPy.
        write_wav(audio_buffer, 16000, audio_int16)

        # Regresar el cursor al inicio del buffer para que pueda ser leído
        audio_buffer.seek(0)

        logging.info("Audio extraído a buffer en memoria exitosamente.")
        return audio_buffer
    except Exception as e:
        logging.error(f"Error al convertir el video a buffer de audio para '{video_path}': {e}")
        return None
    finally:
        if audio_clip:
            audio_clip.close()
        if video_clip:
            video_clip.close()


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [11]:
### Autor Fernando Dorantes Nieto
# -*- coding: utf-8 -*-
import pytesseract
from PIL import Image
import logging
import os

# --- CONFIGURACIÓN DE TESSERACT ---
# Si instalaste Tesseract en una ubicación no estándar, o si no lo agregaste al PATH,
# descomenta la siguiente línea y ajusta la ruta a tu ejecutable tesseract.exe.
# Por ejemplo, en una instalación típica de Windows:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


def extract_text_from_image(image_path: str, language: str = 'spa') -> str | None:
    """
    Extrae texto de un archivo de imagen utilizando Tesseract OCR.

    Args:
        image_path (str): La ruta al archivo de imagen.
        language (str): El código de idioma para Tesseract (ej. 'eng' para inglés, 'spa' para español).

    Returns:
        str | None: El texto extraído de la imagen, o None si ocurre un error.
    """
    if not os.path.exists(image_path):
        logging.error(f"El archivo de imagen no fue encontrado: {image_path}")
        return None

    try:
        logging.info(f"Iniciando extracción de texto (OCR) para la imagen: {os.path.basename(image_path)}")
        # Usamos Pillow (PIL) para abrir la imagen y pytesseract para extraer el texto
        text = pytesseract.image_to_string(Image.open(image_path), lang=language)
        logging.info("Extracción de texto de la imagen completada.")
        return text.strip()
    except Exception as e:
        logging.error(f"Error durante el OCR en el archivo '{image_path}': {e}")
        return None

### Pipeline

In [12]:
### Script principal o el main
### Autor Fernando Dorantes Nieto
# -*- coding: utf-8 -*-
### Fernando Dorantes Nieto
'''
<(*)
  ( >)"
  /|
'''
import os
import shutil
import logging
from docling.document_converter import DocumentConverter


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def convertDocument(filename_string):
    """
    Convierte un documento (como .docx, .pdf) a formato Markdown.

    Args:
        filename_string (str): La ruta al archivo del documento.
    """
    converter = DocumentConverter()
    doc = converter.convert(filename_string).document
    md=  doc.export_to_markdown()
    return md







def process_file_to_text(file_path: str) -> str | None:
    """
    Procesa un archivo para extraer su contenido de texto basado en su tipo.

    Args:
        file_path (str): La ruta al archivo a procesar.

    Returns:
        str | None: El texto extraído del archivo, o None si falla la extracción.
    """
    category = get_file_category(file_path)
    filename = os.path.basename(file_path)
    logging.info(f"Procesando archivo '{filename}' en la categoría: {category}")

    try:
        if category == "audio":
            return transcribe_audio_file(file_path)
        elif category == "video":
            return transcribe_video_to_audio_to_text(file_path)
        elif category == "texto":
            # convertDocument maneja .docx, .pdf, .pptx, etc.
            return convertDocument(file_path)
        elif category == "imagen":
            return extract_text_from_image(file_path, language='spa')
        else:
            logging.warning(f"Categoría de archivo '{category}' no soportada para '{filename}'.")
            return None
    except Exception as e:
        logging.error(f"Error al procesar el archivo '{filename}': {e}")
        return None


def pipeline_consar(file_path: str) -> str | None:
    """
    Función principal que toma la ruta de un archivo, detecta su tipo,
    lo convierte a texto simple y devuelve el resultado.

    Args:
        file_path (str): La ruta completa al archivo a procesar.

    Returns:
        str: El contenido del archivo como texto simple si la conversión es exitosa.
             Si la conversión falla, devuelve la ruta original del archivo (`file_path`).
             Devuelve `None` solo si el archivo de entrada no existe.
    """
    if not os.path.exists(file_path):
        logging.error(f"El archivo no fue encontrado: {file_path}")
        return None

    logging.info(f"Iniciando pipeline para el archivo: {os.path.basename(file_path)}")
    extracted_text = process_file_to_text(file_path)

    if extracted_text:
        logging.info(f"Pipeline completado. Se extrajo texto del archivo.")
        return extracted_text
    else:
        logging.warning(f"No se pudo extraer texto del archivo '{os.path.basename(file_path)}'. Devolviendo la ruta original.")
        return file_path


def process_folder_and_move_files(input_folder: str, success_folder: str, failed_folder: str):
    """
    Procesa todos los archivos de una carpeta, convierte los exitosos a .txt y mueve los fallidos.

    Args:
        input_folder (str): La carpeta con los archivos originales.
        success_folder (str): La carpeta donde se guardarán los .txt de las conversiones exitosas.
        failed_folder (str): La carpeta donde se moverán los archivos que no se pudieron convertir.
    """
    os.makedirs(success_folder, exist_ok=True)
    os.makedirs(failed_folder, exist_ok=True)

    if not os.path.isdir(input_folder):
        logging.error(f"La carpeta de entrada no existe: {input_folder}")
        return

    logging.info(f"Iniciando procesamiento de la carpeta: {input_folder}")

    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        if not os.path.isfile(file_path):
            continue

        result = pipeline_consar(file_path)

        # Comprobamos si el resultado es el texto extraído o la ruta del archivo original
        if result and result != file_path:
            # --- ÉXITO: El resultado es el texto extraído ---
            base_filename = os.path.splitext(filename)[0]
            output_txt_path = os.path.join(success_folder, f"{base_filename}.txt")
            try:
                with open(output_txt_path, "w", encoding="utf-8") as f:
                    f.write(result)
                logging.info(f"Éxito: '{filename}' -> Guardado en '{output_txt_path}'")
            except Exception as e:
                logging.error(f"Error al guardar el archivo de texto '{output_txt_path}': {e}")
        else:
            # --- FALLO: El resultado es la ruta del archivo o None ---
            failed_path = os.path.join(failed_folder, filename)
            try:
                shutil.move(file_path, failed_path)
                logging.warning(f"Fallo: '{filename}' -> Movido a '{failed_path}'")
            except Exception as e:
                logging.error(f"Error al mover el archivo fallido '{filename}': {e}")

    logging.info("Procesamiento de la carpeta completado.")

"""
if __name__ == '__main__':
    # --- Ejemplo de uso con la nueva función orquestadora ---

    # 1. Define tus carpetas
    INPUT_DIR = "C:/ruta/a/tus/archivos_de_entrada"
    SUCCESS_DIR = "C:/ruta/a/tus/archivos_convertidos"
    FAILED_DIR = "C:/ruta/a/tus/archivos_fallidos"

    # 2. Llama a la función principal
    process_folder_and_move_files(INPUT_DIR, SUCCESS_DIR, FAILED_DIR)
"""

'\nif __name__ == \'__main__\':\n    # --- Ejemplo de uso con la nueva función orquestadora ---\n    \n    # 1. Define tus carpetas\n    INPUT_DIR = "C:/ruta/a/tus/archivos_de_entrada"\n    SUCCESS_DIR = "C:/ruta/a/tus/archivos_convertidos"\n    FAILED_DIR = "C:/ruta/a/tus/archivos_fallidos"\n\n    # 2. Llama a la función principal\n    process_folder_and_move_files(INPUT_DIR, SUCCESS_DIR, FAILED_DIR)\n'

### Ejecutando Pipeline

In [13]:
INPUT_DIR = "/content/drive/MyDrive/RAG_CONSAR/archivos_descargados"
SUCCESS_DIR = "/content/drive/MyDrive/RAG_CONSAR/archivos_convertidos_a_texto"
FAILED_DIR = "/content/drive/MyDrive/RAG_CONSAR/archivos_fallidos_convertir"


In [ ]:
process_folder_and_move_files(INPUT_DIR, SUCCESS_DIR, FAILED_DIR)

[INFO] 2025-11-19 18:49:14,499 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-19 18:49:14,516 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-19 18:49:14,518 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-19 18:49:14,604 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-19 18:49:14,607 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-11-19 18:49:14,608 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-11-19 18:49:14,653 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-19 18:49:14,684 [RapidOCR] download_file.py:60: File exists and is valid: /usr/loc

Creando archivo temporal
MoviePy - Writing audio in temp.wav


MoviePy - Done.
Borrando archivo temporal


Se truncaron las últimas líneas 5000 del resultado de transmisión.
                    511,
                    511,
                    511,
                    511,
                    511,
                    511,
                    511,
                    511,
                    511,
                    284,
                    284,
                    577,
                    577,
                    577,
                    454,
                    941,
                    738,
                    682,
                    682,
                    738,
                    624,
                    568,
                    738,
                    738,
                    340,
                    398,
                    738,
                    624,
                    909,
                    738,
                    738,
                    568,
                    738,
                    682,
                    568,
                    624,
                    738,
        

## TEST

In [ ]:
import sys
import os
import unicodedata
import re

## Función para remover acentos

def remove_accents(input_str):
    if not isinstance(input_str, str):
        raise TypeError("Input must be a string.")
    normalized = unicodedata.normalize('NFD', input_str)

    without_accents = ''.join(
        char for char in normalized
        if unicodedata.category(char) != 'Mn'
    )
    return without_accents


def clean_string(string_to_clean, with_underscore=False):
  string_cleaned = re.sub(r'_', ' ', string_to_clean)
  ## Cambiar acentos por letras sin acentos, también aplica la ñ
  string_cleaned = remove_accents(string_cleaned)
  ## Remover caracteres raros
  string_cleaned = re.sub(r'[^\w\s]', '', string_cleaned)
  ## Convert to UTF8
  string_cleaned = string_cleaned.encode('utf-8').decode('utf-8')
  string_cleaned = re.sub(r'\s+', ' ', string_cleaned)
  ### Remover espacios al inicio y al final
  string_cleaned = string_cleaned.strip()
  if with_underscore:
    return  string_cleaned.replace(' ', '_').lower()
  else:
    return string_cleaned.lower()


In [ ]:
drive_folder_pdf_to_upload = '/content/drive/MyDrive/RAG_CONSAR/archivos_descargados'
drive_folder_txt_to_upload = '/content/drive/MyDrive/RAG_CONSAR/archivos_descargados'
drive_folder_credentials = '/content/drive/MyDrive/keys'

In [ ]:
async def descargar_pdfs_consar(dir_to_download_files, existing_df=None):
    #download_dir = "./descargas_consar"
    download_dir = dir_to_download_files
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    # Prepare a set of already downloaded file identifiers for quick lookup
    # We will use the generated filename as the identifier, e.g., "NOMBRE_0.pdf"
    ### Verificar que el nombre de la path exista
    downloaded_file_names = set()
    if existing_df is not None and not existing_df.empty:
        downloaded_file_names = set(existing_df['archivo_descargado'].dropna().tolist())

    datos = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            accept_downloads=True,
            viewport={'width': 1920, 'height': 1080}
        )
        page = await context.new_page()
        print("Navegando a la página...")
        await page.goto('https://www.consar.gob.mx/gobmx/aplicativo/SINORconsar#')
        await page.wait_for_selector('table', timeout=30000)
        print("Tabla detectada, extrayendo datos...")

        filas = await page.query_selector_all('table tbody tr')

        for i, fila in enumerate(filas):
            try:
                celdas = await fila.query_selector_all('td')
                if len(celdas) >= 3:
                    nombre = (await celdas[0].inner_text()).strip()
                    print('Nombre')
                    tipo = (await celdas[1].inner_text()).strip()
                    fecha_dof = (await celdas[2].inner_text()).strip()

                    expected_filename = f"{nombre}_{i}.pdf"
                    ruta_completa = os.path.join(download_dir, expected_filename)

                    # Check if already downloaded and file exists
                    if expected_filename in downloaded_file_names and os.path.exists(ruta_completa):
                        print(f"Saltando: '{nombre}' (ya descargado como '{expected_filename}')")
                        datos.append({
                            'nombre': nombre,
                            'tipo': tipo,
                            'fecha_publicacion_dof': fecha_dof,
                            'archivo_descargado': expected_filename,
                            'ruta_completa': ruta_completa
                        })
                        continue # Skip to next row

                    print(f"Procesando: {nombre}")
                    archivo_descargado = None
                    try:
                        enlace_nombre = await celdas[0].query_selector('a[href*=".pdf"], a[href*="download"]')
                        if enlace_nombre:
                            async with page.expect_download(timeout=60000) as download_info:
                                await enlace_nombre.click()
                                await page.wait_for_timeout(500)
                            download = await download_info.value
                            if download.suggested_filename.lower().endswith('.pdf'):
                                archivo_descargado = expected_filename
                                await download.save_as(ruta_completa)
                                print(f"  ✓ Descargado desde nombre: {archivo_descargado}")

                    except Exception as e:
                        print(f"  ✗ Error descargando desde nombre para '{nombre}': {e}")

                    if not archivo_descargado:
                        try:
                            # Try to find a download button/link more generically in the row
                            boton_descargar = await fila.query_selector('button:has-text("Descargar"), a:has-text("Descargar"), a[href*=".pdf"], a[href*="download"]')
                            if boton_descargar:
                                async with page.expect_download(timeout=60000) as download_info: # Increased timeout
                                    await boton_descargar.click()
                                    await page.wait_for_timeout(500) # Small wait
                                download = await download_info.value
                                if download.suggested_filename.lower().endswith('.pdf'):
                                    archivo_descargado = expected_filename
                                    await download.save_as(ruta_completa)
                                    print(f"  ✓ Descargado desde botón: {archivo_descargado}")

                        except Exception as e:
                            print(f"  ✗ Error descargando desde botón para '{nombre}': {e}")

                    datos.append({
                        'nombre': nombre,
                        'tipo': tipo,
                        'fecha_publicacion_dof': fecha_dof,
                        'archivo_descargado': archivo_descargado,
                        'ruta_completa': ruta_completa if archivo_descargado else None
                    })

            except Exception as e:
                print(f"Error procesando fila {i} ('{nombre if 'nombre' in locals() else 'N/A'}'): {e}")
                continue

        await browser.close()
    return datos, download_dir

def crear_dataframe_y_guardar(datos, download_dir):
    # Crear DataFrame
    df = pd.DataFrame(datos)

    # Guardar DataFrame como CSV
    csv_path = os.path.join(download_dir, "metadata_documentos.csv")
    df.to_csv(csv_path, index=False, encoding='utf-8')

    print(f"\n✅ Proceso completado:")
    print(f"   - Documentos procesados: {len(df)}")
    print(f"   - PDFs descargados: {df['archivo_descargado'].notna().sum()}")
    print(f"   - CSV guardado en: {csv_path}")
    print(f"   - PDFs guardados en: {download_dir}")

    return df

# Función para subir a Google Drive (opcional)
def subir_a_drive(local_folder, drive_folder_id=None):
    """
    Función para subir archivos a Google Drive
    Requiere: pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
    """
    try:

        # Configurar autenticación (necesitas configurar las credenciales OAuth 2.0)
        # service = build('drive', 'v3', credentials=creds)

        # Subir archivos
        for filename in os.listdir(local_folder):
            file_path = os.path.join(local_folder, filename)
            if os.path.isfile(file_path):
                file_metadata = {
                    'name': filename,
                    'parents': [drive_folder_id] if drive_folder_id else []
                }
                media = MediaFileUpload(file_path, resumable=True)
                # file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
                print(f"Subido a Drive: {filename}")

    except ImportError:
        print("Bibliotecas de Google Drive no instaladas. Instala con: pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib")
    except Exception as e:
        print(f"Error subiendo a Drive: {e}")

In [ ]:
import pandas as pd
import io
import os
import logging
from playwright.sync_api import sync_playwright

# --- IMPORTACIONES PARA GOOGLE DRIVE ---
# pip install playwright pandas google-api-python-client google-auth-httplib2 google-auth-oauthlib PyMuPDF
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from google.auth.transport.requests import Request

# --- IMPORTACIÓN PARA PDF A TEXTO ---
import fitz # PyMuPDF

# -----------------------------------------------------------------
# --- CONFIGURACIÓN Y AUTENTICACIÓN ---
# -----------------------------------------------------------------

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s'
                    )

# --- CONFIGURACIÓN DRIVE (TOKEN PERSISTENTE) ---
SCOPES = ['https://www.googleapis.com/auth/drive']
TOKEN_FILE = 'token.json'
CREDENTIALS_FILE = 'credentials.json'

# --- CONFIGURACIÓN CARPETAS DRIVE ---
PDF_FOLDER_NAME = "CONSAR_PDF_FUENTES" # Carpeta para los PDFs originales
TXT_FOLDER_NAME = "CONSAR_TXT_RAG"    # Carpeta para los TXT procesados (fuente de NotebookLM)


# -----------------------------------------------------------------
# --- MANEJO DE CARPETAS Y DEDUPLICACIÓN DE GOOGLE DRIVE ---
# -----------------------------------------------------------------

def find_or_create_folder(service, folder_name):
    """
    Busca una carpeta por nombre. Si no existe, la crea. Retorna el ID de la carpeta.
    """
    try:
        # Buscar la carpeta
        query = (
            f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}' and trashed=false"
        )
        results = service.files().list(q=query,
                                       spaces='drive',
                                       fields='files(id, name)').execute()

        items = results.get('files', [])
        if items:
            folder_id = items[0]['id']
            logging.info(f"Carpeta '{folder_name}' encontrada. ID: {folder_id}")
            return folder_id

        # Si no se encuentra, crearla
        logging.info(f"Carpeta '{folder_name}' no encontrada. Creando...")
        file_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        folder = service.files().create(body=file_metadata, fields='id').execute()
        folder_id = folder.get('id')
        logging.info(f"Carpeta '{folder_name}' creada. ID: {folder_id}")
        return folder_id

    except Exception as e:
        logging.error(f"Error al buscar o crear la carpeta '{folder_name}': {e}")
        return None

def get_existing_drive_filenames(service, folder_id):
    """
    Lista todos los nombres de archivos en una carpeta de Drive para la deduplicación.
    """
    try:
        # q: consulta para buscar archivos en la carpeta específica (parents) y que no estén en la papelera (trashed=false)
        query = f"'{folder_id}' in parents and trashed=false"
        results = service.files().list(q=query,
                                       spaces='drive',
                                       fields='files(name)').execute()

        # Crea un conjunto de nombres de archivo para búsquedas eficientes
        existing_files = {item['name'] for item in results.get('files', [])}
        logging.info(f"Encontrados {len(existing_files)} archivos existentes en la carpeta de destino TXT.")
        return existing_files

    except Exception as e:
        logging.error(f"Error al listar archivos en Google Drive: {e}")
        return set() # Retorna un conjunto vacío para continuar

def upload_to_gdrive(service, folder_id, file_name, file_content_bytes, mime_type):
    """
    Sube un archivo (como bytes) a una carpeta específica de Google Drive.
    """
    try:
        file_metadata = {
            'name': file_name,
            'parents': [folder_id]
        }

        # Crea un stream de bytes en memoria
        media = io.BytesIO(file_content_bytes)

        # Prepara el objeto MediaIoBaseUpload
        media_upload = MediaIoBaseUpload(media,
                                         mimetype=mime_type,
                                         resumable=True)

        # Sube el archivo
        file = service.files().create(body=file_metadata,
                                      media_body=media_upload,
                                      fields='id, webViewLink').execute()

        logging.info(f"Archivo '{file_name}' subido. Tipo: {mime_type}")
        # Retorna el enlace para verlo en la web
        return file.get('webViewLink')

    except Exception as e:
        logging.error(f"Error al subir el archivo '{file_name}': {e}")
        return None

# -----------------------------------------------------------------
# --- FUNCIÓN DE CONVERSIÓN (PyMuPDF) ---
# -----------------------------------------------------------------

def pdf_bytes_to_text(pdf_bytes):
    """
    Extrae texto de un PDF en memoria usando PyMuPDF (fitz).
    NOTA: Este método maneja PDFs nativos. Para PDFs que son solo imágenes,
    necesitarías una API de OCR externa (como Docling o DeepSeek), lo cual
    requiere claves de API y manejo de errores adicional.
    """
    try:
        with fitz.open(stream=pdf_bytes, filetype="pdf") as doc:
            texto_completo = ""
            for page in doc:
                texto_completo += page.get_text("text") + "\n\n"
        return texto_completo.strip()
    except Exception as e:
        logging.error(f"Error extrayendo texto con PyMuPDF: {e}")
        return None

# -----------------------------------------------------------------
# --- LÓGICA DE SCRAPING CON PLAYWRIGHT ---
# -----------------------------------------------------------------

def scrape_consar_data(url):
    """
    Extrae los datos, descarga el PDF, lo convierte a TXT y sube ambos a Drive.
    """

    BASE_URL = "https://www.consar.gob.mx"

    with sync_playwright() as p:
        # Usamos chromium en modo headless para GitHub Actions
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()

        try:
            # --- 1. CONFIGURACIÓN INICIAL DE DRIVE ---
            gdrive_service = authenticate_gdrive()
            if not gdrive_service:
                return pd.DataFrame() # Salir si la autenticación falla

            # Obtener IDs de carpetas (crear si no existen)
            pdf_folder_id = find_or_create_folder(gdrive_service, PDF_FOLDER_NAME)
            txt_folder_id = find_or_create_folder(gdrive_service, TXT_FOLDER_NAME)

            if not pdf_folder_id or not txt_folder_id:
                logging.error("No se pudo obtener el ID de una o ambas carpetas de Drive.")
                return pd.DataFrame()

            # Obtener lista de archivos TXT existentes para deduplicación
            existing_txt_files = get_existing_drive_filenames(gdrive_service, txt_folder_id)

            # --- 2. SCRAPING DE LA TABLA ---
            logging.info(f"Navegando a: {url}")
            page.goto(url, wait_until="networkidle")

            table_selector = "table#myTable"
            page.wait_for_selector(table_selector, timeout=30000)
            logging.info("Tabla de documentos encontrada.")

            scraped_data = []
            rows = page.locator(f"{table_selector} tbody tr").all()
            logging.info(f"Se encontraron {len(rows)} filas.")

            for i, row in enumerate(rows):
                cols = row.locator('td').all()
                if len(cols) < 7: continue

                nombre = cols[0].inner_text().strip()
                tipo = cols[1].inner_text().strip()
                fecha_dof = cols[4].inner_text().strip()
                download_url = None

                # Lógica para encontrar download_url (similar al script anterior)
                nombre_link = cols[0].query_selector('a')
                if nombre_link:
                    href = nombre_link.get_attribute('href')
                    if href and 'pdf' in href.lower():
                        download_url = href

                if not download_url:
                    descargar_link = cols[6].query_selector('a')
                    if descargar_link:
                        href = descargar_link.get_attribute('href')
                        if href:
                            download_url = href

                if download_url:
                    if not download_url.startswith('http'):
                        download_url = f"{BASE_URL}{download_url}"
                    download_url = download_url.strip()

                pdf_drive_link = None
                txt_drive_link = None

                # --- 3. LÓGICA DE PROCESAMIENTO Y SUBIDA ---
                if download_url:

                    # Pre-calcular nombres de archivos
                    clean_name = "".join(c for c in nombre if c.isalnum() or c in (' ', '_')).rstrip()
                    clean_date = fecha_dof.replace("/", "-")

                    # Nombres de archivo final:
                    pdf_filename = f"{clean_date}_{tipo}_{clean_name}.pdf".replace(" ", "_").replace("__", "_")
                    txt_filename = f"{clean_date}_{tipo}_{clean_name}.txt".replace(" ", "_").replace("__", "_")

                    # --- COMPROBACIÓN DE DUPLICADOS (Usando el nombre del archivo TXT) ---
                    if txt_filename in existing_txt_files:
                        logging.info(f"[Fila {i+1}] Saltando: '{txt_filename}' ya existe. (Deduplicación activa)")
                    else:
                        logging.info(f"[Fila {i+1}] Procesando nuevo archivo: {nombre}")

                        try:
                            # a) Descargar el PDF en memoria
                            pdf_page = browser.new_page()
                            pdf_response = pdf_page.goto(download_url, timeout=30000)
                            pdf_content_bytes = pdf_response.body()
                            pdf_page.close()

                            # b) Subir el PDF original a la carpeta de FUENTES
                            pdf_drive_link = upload_to_gdrive(
                                gdrive_service,
                                pdf_folder_id,
                                pdf_filename,
                                pdf_content_bytes,
                                mime_type='application/pdf'
                            )

                            # c) Convertir el PDF a texto simple
                            document_text = pdf_bytes_to_text(pdf_content_bytes)

                            if document_text:
                                text_content_bytes = document_text.encode('utf-8')

                                # d) Subir el archivo TXT a la carpeta RAG (fuente de NotebookLM)
                                txt_drive_link = upload_to_gdrive(
                                    gdrive_service,
                                    txt_folder_id,
                                    txt_filename,
                                    text_content_bytes,
                                    mime_type='text/plain'
                                )
                            else:
                                logging.warning(f"No se pudo extraer texto del PDF: {nombre}")

                        except Exception as e:
                            logging.error(f"Error en el ciclo de descarga/conversión/subida de {nombre}: {e}")
                            pass

                scraped_data.append({
                    "nombre": nombre,
                    "tipo": tipo,
                    "fecha_publicacion_dof": fecha_dof,
                    "download_url": download_url,
                    "pdf_drive_link": pdf_drive_link,
                    "txt_drive_link": txt_drive_link # Este es el link a conectar con NotebookLM
                })

            final_df = pd.DataFrame(scraped_data)
            return final_df

        except Exception as e:
            logging.error(f"Ocurrió un error grave en el scraping: {e}")
            page.screenshot(path="error_screenshot_scrape.png")
            return pd.DataFrame()

        finally:
            browser.close()

# -----------------------------------------------------------------
# --- EJECUCIÓN PRINCIPAL ---
# -----------------------------------------------------------------

if __name__ == "__main__":

    print("Iniciando scraper de CONSAR para RAG y subida a Google Drive...")

    target_url = "https://www.consar.gob.mx/gobmx/aplicativo/SINORconsar#"
    df = scrape_consar_data(target_url)

    if not df.empty:
        print("\n--- Resumen de Archivos Procesados (Primeros 10) ---")
        print(df[['nombre', 'txt_drive_link']].head(10))

        df.to_csv("consar_sinor_data_summary_drive.csv", index=False, encoding='utf-8-sig')
        print(f"\nResumen de datos guardado en 'consar_sinor_data_summary_drive.csv'")
    else:
        print("No se extrajeron datos.")